In [1]:
import pandas as pd
import numpy as np
import json
import re


## Load Data

Load product metadata and reviews from CSV files.


In [2]:

products_df = pd.read_csv('../data/amazon_meta.csv')
reviews_df = pd.read_csv('../data/amazon_reviews.csv')


## Data Exploration

Analyze missing values in both datasets to understand data quality.


In [3]:
missing = reviews_df.isna().sum()
missing_percent = (missing / len(reviews_df)) * 100
missing_summary = pd.DataFrame({
    "missing_count": missing,
    "missing_%": missing_percent.round(2)
}).sort_values(by="missing_%", ascending=False)
print("\n Missing Values Summary:")

display(missing_summary)


 Missing Values Summary:


,missing_count,missing_%
title,804,0.02
text,1033,0.02
Unnamed: 0,0,0.00
rating,0,0.00
images,0,0.00
asin,0,0.00
parent_asin,0,0.00
user_id,0,0.00
timestamp,0,0.00
helpful_vote,0,0.00


In [4]:
# 50% of products missing price
missing = products_df.isna().sum()
missing_percent = (missing / len(products_df)) * 100
missing_summary = pd.DataFrame({
    "missing_count": missing,
    "missing_%": missing_percent.round(2)
}).sort_values(by="missing_%", ascending=False)
print("\n Missing Values Summary:")
display(missing_summary)


 Missing Values Summary:


,missing_count,missing_%
bought_together,137269,100.00
author,137007,99.81
subtitle,136919,99.75
price,75261,54.83
main_category,11036,8.04
store,4375,3.19
title,9,0.01
Unnamed: 0,0,0.00
average_rating,0,0.00
images,0,0.00


## Data Cleaning

In [5]:
def clean_price(x):
    s = str(x).strip().lower()
    if pd.isna(x) or s in {'', 'nan', 'none', 'n/a', 'free'}:
        return np.nan
    
    nums = [float(n) for n in re.findall(r'\d*\.\d+|\d+', s.replace(',', ''))]
    
    return int(round(sum(nums) / len(nums))) if nums else np.nan

products_df['price'] = products_df['price'].apply(clean_price)

products_df['price'] = products_df['price'].fillna(
    products_df.groupby('main_category')['price'].transform('mean')
).fillna(
    products_df['price'].median()
).astype(int)

In [6]:
reviews_df = reviews_df[['parent_asin', 'rating', 'user_id', 'title', 'text']]
reviews_df = reviews_df.rename(columns={'title': 'review_title'})


products_df = products_df[['parent_asin', 'price', 'title', 'categories']]
products_df = products_df.rename(columns={'title': 'product_title'})

In [7]:
print("new product_df columns: ", products_df.columns)
print("new reviews_df columns: ", reviews_df.columns)


new product_df columns:  Index(['parent_asin', 'price', 'product_title', 'categories'], dtype='object')
new reviews_df columns:  Index(['parent_asin', 'rating', 'user_id', 'review_title', 'text'], dtype='object')


In [8]:
reviews_df['review_title'] = reviews_df['review_title'].fillna('')
reviews_df['text'] = reviews_df['text'].fillna('')

products_df['product_title'] = products_df['product_title'].fillna('')
products_df['categories'] = products_df['categories'].fillna('')

median_price = products_df['price'].median().astype(float)
products_df['price'] = products_df['price'].fillna(median_price)

## Test the merge first

In [9]:
test_products = products_df.sample(n=5).copy() 

target_ids = test_products['parent_asin'].unique()

test_reviews = reviews_df[reviews_df['parent_asin'].isin(target_ids)].copy()

test_merged = test_products.merge(
    test_reviews, 
    on='parent_asin', 
    how='inner'
)

# 5. Verify the results
print(f"Products sample shape: {test_products.shape}")
print(f"Reviews sample shape: {test_reviews.shape}")
print(f"Merged shape: {test_merged.shape}")
print("-" * 30)

len(test_merged)


Products sample shape: (5, 4)
Reviews sample shape: (29, 5)
Merged shape: (29, 8)
------------------------------


29

In [10]:
merged_df = products_df.merge(
    reviews_df, 
    on='parent_asin', 
    how='inner'
)  

merged_df.to_csv('../data/combined_data.csv', index=False)


In [11]:
merged_df.head()

,parent_asin,price,product_title,categories,rating,user_id,review_title,text
0,B000FH0MHO,51,Dash 8-300 Professional Add-On,"['Video Games', 'PC', 'Games']",5.0,AE7URMTTXBBAEQRQFB7OQINN6L5Q,Dash 8-300,I bought the Dash 8-300 FS2002/2004 several ye...
1,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",5.0,AGRAC4UNOZPFIJNKBW6BYZAV7A4Q,Brings back memories,my fiance played this game a long time ago and...
2,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",1.0,AGFFZA4CEPZMVLMZR4DCQKCNBB7Q,Total waste of money and disappointment!,Came missing one disk which makes the game tot...
3,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",4.0,AFJZI2YJZ55Q2ILQQ4L452BRB26A,"a hauntingly gross, fun gore-ride",I adored this game and its sequel. Nothing to...
4,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",4.0,AFJGC4QR55F3OHRKWCQRZCWNWRSA,My childhood,"I love this game, I mean, its old and all but ..."


# Combine the text into a single field

In [12]:
def clean_text(s):
    s = str(s).lower()                        
    s = re.sub(r"[^\w\s]", " ", s)           
    s = re.sub(r"\d+", " ", s)                
    s = re.sub(r"\s+", " ", s).strip()        
    return s


In [14]:
def merge_text_columns(row):
    parts = []

    fields = ["product_title", "categories", "review_title", "text"]

    for col in fields:
        value = row.get(col)

        if pd.isna(value):
            continue
        
        if isinstance(value, (list, dict)):
            try:
                value = json.dumps(value)
            except:
                value = str(value)
        
        parts.append(str(value))

    return " ".join(parts)
prep_df = merged_df.copy()
prep_df["merged_text"] = merged_df.apply(merge_text_columns, axis=1)
prep_df["merged_text"] = prep_df["merged_text"].apply(clean_text)
prep_df = prep_df.drop(columns=["product_title", "categories", "review_title", "text"])

KeyboardInterrupt: 

In [15]:

prep_df = merged_df.copy()

for col in ["product_title", "categories", "review_title", "text"]:
    prep_df[col] = prep_df[col].astype(str).replace("nan", "")
prep_df["merged_text"] = (
      prep_df["product_title"] + " "
    + prep_df["categories"] + " "
    + prep_df["review_title"] + " "
    + prep_df["text"]
)

def normalize_value(col):
    return col.apply(lambda x: json.dumps(x) if isinstance(x, (list,dict)) else str(x) if pd.notna(x) else "")
    
prep_df["product_title"] = normalize_value(prep_df["product_title"])
prep_df["categories"]    = normalize_value(prep_df["categories"])
prep_df["review_title"]  = normalize_value(prep_df["review_title"])
prep_df["text"]          = normalize_value(prep_df["text"])

prep_df["merged_text"] = prep_df["product_title"] + " " + prep_df["categories"] + " " + prep_df["review_title"] + " " + prep_df["text"]


In [16]:
prep_df.to_csv('../data/prep.csv', index=False)
prep_df.head()

,parent_asin,price,product_title,categories,rating,user_id,review_title,text,merged_text
0,B000FH0MHO,51,Dash 8-300 Professional Add-On,"['Video Games', 'PC', 'Games']",5.0,AE7URMTTXBBAEQRQFB7OQINN6L5Q,Dash 8-300,I bought the Dash 8-300 FS2002/2004 several ye...,"Dash 8-300 Professional Add-On ['Video Games',..."
1,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",5.0,AGRAC4UNOZPFIJNKBW6BYZAV7A4Q,Brings back memories,my fiance played this game a long time ago and...,Phantasmagoria: A Puzzle of Flesh ['Video Game...
2,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",1.0,AGFFZA4CEPZMVLMZR4DCQKCNBB7Q,Total waste of money and disappointment!,Came missing one disk which makes the game tot...,Phantasmagoria: A Puzzle of Flesh ['Video Game...
3,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",4.0,AFJZI2YJZ55Q2ILQQ4L452BRB26A,"a hauntingly gross, fun gore-ride",I adored this game and its sequel. Nothing to...,Phantasmagoria: A Puzzle of Flesh ['Video Game...
4,B00069EVOG,51,Phantasmagoria: A Puzzle of Flesh,"['Video Games', 'PC', 'Games']",4.0,AFJGC4QR55F3OHRKWCQRZCWNWRSA,My childhood,"I love this game, I mean, its old and all but ...",Phantasmagoria: A Puzzle of Flesh ['Video Game...
